## Kelvin-Helmholtz Notebook

First, we set up the simulation parameters and specify the species involved. 

In [1]:
import em2d as zpic

#Simulation box
nx  = [ 200, 200]
box = [ 10.0, 10.0 ]

#Time parameters
dt = 0.014
tmax = 200.0

#Displays only part of the box (required for phasespace data)
sim_range = [[0,box[0]],[-0.3,0.3]]

#Stores zdf data for every ndump timesteps
ndump = 10

Next we need to describe the particle species in the simulation. In this example, we are using 4 species for both the ions and electrons:

In [2]:
import numpy as np

# Particles per cell
ppc = [6,6]

#Plasma KHI Velocity Shear Positions
x1 = box[0] / 4
x2 = box[0] * (3 / 4)

#Center Density Function
def nx1(x) :
    return np.heaviside((x-x1), 0) - np.heaviside((x-x2), 1);

#Sides Density Function
def nx2(x) :
    return np.heaviside((x1-x), 1) + np.heaviside((x-x2), 1);

#Initialize densities (default normalization: n = 1)
density1 = zpic.Density( type = "custom", custom_x = nx1)
density2 = zpic.Density( type = "custom", custom_x = nx2)

#Mass-to-Charge Ratio
m_q = [-1.0, 25]

# Initialize fluid and thermal velocities 
ufl0 = np.array([0.0,    0.1,  0.0])
ufl1 = -1 * ufl0
eth = 0.005
uth0 = [np.sqrt(abs(2*eth/m_q[0])),np.sqrt(abs(2*eth/m_q[0])),np.sqrt(abs(2*eth/m_q[0]))]
uth1 = [np.sqrt(abs(2*eth/m_q[1])),np.sqrt(abs(2*eth/m_q[1])),np.sqrt(abs(2*eth/m_q[1]))]
ufl_a = [list(ufl0), list(ufl1)]

#ZDF labels for all the species
names = ["up electrons", "up ions", "down electrons", "down ions"]
speciesn = []

#Initialize species
speciesn.append( zpic.Species( names[0], m_q[0], ppc, ufl = list(ufl_a[0]) , uth = uth0, density = density1 ))
speciesn.append( zpic.Species( names[1], m_q[1], ppc, ufl = list(ufl_a[0]), uth = uth1, density = density1 ))
speciesn.append( zpic.Species( names[2], m_q[0], ppc, ufl = list(ufl_a[1]) , uth = uth0, density = density2 ))
speciesn.append( zpic.Species( names[3], m_q[1], ppc, ufl = list(ufl_a[1]), uth = uth1, density = density2 ))

#Gif Animation Description 
gif_description = 'm to e = ' + str(m_q) + '_ufl = ' + str(list((ufl_a))) + '_eth = ' + str(eth) + '_ppc = ' + str(ppc) + '_box = ' + str(box) + '_nx = ' + str(nx) + '_dt = ' + str(dt) + '.gif'

## Writing diagnostic output to disk

You can run ZPIC inside the notebook (or any interactive Python/iPython session) and access all the simulation data directly in memory, without writing any output to disk, as described in other notebooks in this folder. For most situations this is the recommended way of using the code. However, if you your simulation takes a long time to compute, you may want to write diagnostic information to disk for post-processing later.

To do this you must define the required diagnostics in a python function that accepts as a single argument a simulation object. This routine will be called once per iteration, and it can access global variables defined in the Python code, e.g.:

In [3]:
def rep( sim ):
    # sim.n has the current simulation iteration
    if (sim.n % ndump == 0):
        
        #Electric Field Data
        sim.emf.report("E", 0) #x
        sim.emf.report("E", 1) #y
        sim.emf.report("E", 2) #z
        
        #Magnetic Field Data
        sim.emf.report("B", 0)
        sim.emf.report("B", 1)
        sim.emf.report("B", 2)
        
        #Particle Position and Velocity Data
        speciesn[0].report("particles")
        speciesn[1].report("particles")
        speciesn[2].report("particles")
        speciesn[3].report("particles")
        
        #Charge Density Data
        speciesn[0].report("charge")
        speciesn[1].report("charge")
        speciesn[2].report("charge")
        speciesn[3].report("charge")
        
        #Phasespace Density Data
        speciesn[0].report("pha", quants = ["x1", "u2"], pha_nx = nx, pha_range = sim_range)
        speciesn[1].report("pha", quants = ["x1", "u2"], pha_nx = nx, pha_range = sim_range)
        speciesn[2].report("pha", quants = ["x1", "u2"], pha_nx = nx, pha_range = sim_range)
        speciesn[3].report("pha", quants = ["x1", "u2"], pha_nx = nx, pha_range = sim_range)
        
        speciesn[0].report("pha", quants = ["x2", "u1"], pha_nx = nx, pha_range = sim_range)
        speciesn[1].report("pha", quants = ["x2", "u1"], pha_nx = nx, pha_range = sim_range)
        speciesn[2].report("pha", quants = ["x2", "u1"], pha_nx = nx, pha_range = sim_range)
        speciesn[3].report("pha", quants = ["x2", "u1"], pha_nx = nx, pha_range = sim_range)
        
        speciesn[0].report("pha", quants = ["x2", "u2"], pha_nx = nx, pha_range = sim_range)
        speciesn[1].report("pha", quants = ["x2", "u2"], pha_nx = nx, pha_range = sim_range)
        speciesn[2].report("pha", quants = ["x2", "u2"], pha_nx = nx, pha_range = sim_range)
        speciesn[3].report("pha", quants = ["x2", "u2"], pha_nx = nx, pha_range = sim_range)
        
        speciesn[0].report("pha", quants = ["x1", "u1"], pha_nx = nx, pha_range = sim_range)
        speciesn[1].report("pha", quants = ["x1", "u1"], pha_nx = nx, pha_range = sim_range)
        speciesn[2].report("pha", quants = ["x1", "u1"], pha_nx = nx, pha_range = sim_range)
        speciesn[3].report("pha", quants = ["x1", "u1"], pha_nx = nx, pha_range = sim_range)

#Initialize simulation
sim = zpic.Simulation( nx, box, dt, species = speciesn, report = rep )

We can now add an external magnetic field before running the simulation. 

In [4]:
#Magnetic Field Function

#ix, iy = cell index by which field is being calculated
#dx, dy = cell size (in sim units)
def extB(ix,dx,iy,dy):
    
    x = ix*dx
    
    a = 0.01
    p = box[1]
    bz = a#*np.sin((2*np.pi / p)*x);

    return [0, 1, bz]

#Initialize a custom magnetic field
ext = zpic.ExternalField(B_type = 'custom', B_custom = extB )

#Initialize a uniform magnetic field
#Bz0 = 1
#ext = zpic.ExternalField(B_type = 'uniform', B_0 = [0.0, 0.0, Bz0])

sim.emf.set_ext_fld( ext )

To run the simulation use the run method, giving the final time as the sole parameter:

In [ ]:
sim.run(tmax)


Running simulation up to t = 200 ...
